# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [2]:
# import libraries
import sys
from sqlalchemy import create_engine
import nltk
from nltk.corpus import stopwords
nltk.download(['punkt', 'wordnet', 'stopwords'])

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\aniyo\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\aniyo\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\aniyo\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
import re
import pickle
import numpy as np
import pandas as pd
from nltk.tokenize import word_tokenize
from nltk.stem.wordnet import WordNetLemmatizer

from sklearn.pipeline import Pipeline
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer, TfidfVectorizer
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV

In [5]:
# load data from database
engine = create_engine('sqlite:///disaster_msg.db')
df = pd.read_sql("SELECT * FROM disaster_msg", engine)
X = df['message']

Y = df.drop(['id', 'message', 'original', 'genre'], axis = 1)

### 2. Write a tokenization function to process your text data

In [4]:
def tokenize(text):
    '''
    A function to normalize, tokenize and stem the given string
    
    Input:
    text : A string containing a message that needs to be processed
    
    Output:
    clean_tok : A list of normalized and stemmed tokens from the given string
    '''
    
    # Normalize text : converting it to lowercase and removing punctuation
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())
    
    # Tokenize text
    words = word_tokenize(text)
    tokens = [w for w in words if w not in stopwords.words('english')]
    
    # Lemmatize
    lemmatizer = WordNetLemmatizer()
    clean_tok = []
    for tok in tokens:
        clean_token = lemmatizer.lemmatize(tok).strip()
        clean_tok.append(clean_token)
    
    return clean_tok

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [30]:
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier())),
    
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [31]:
# Splitting the dataset
X_train, X_test, y_train, y_test = train_test_split(X,Y)

# Training the pipeline
pipeline.fit(X_train, y_train)

C:\Users\aniyo\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\aniyo\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\aniyo\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\aniyo\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\aniyo\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: F

Pipeline(memory=None,
         steps=[('vect',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=<function tokenize at...
                 MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True,
                                                                        class_weight=None,
                                                                        criterion='gini',
                                                                  

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [32]:
y_pred = pipeline.predict(X_test)

In [33]:
print(classification_report(y_test.values, y_pred, target_names = Y.columns.values))
    

                        precision    recall  f1-score   support

               related       0.85      0.91      0.88      4918
               request       0.77      0.46      0.57      1058
                 offer       0.00      0.00      0.00        28
           aid_related       0.73      0.62      0.67      2669
          medical_help       0.58      0.08      0.14       533
      medical_products       0.74      0.12      0.21       312
     search_and_rescue       0.80      0.07      0.13       163
              security       0.40      0.02      0.03       121
              military       0.76      0.06      0.12       204
           child_alone       0.00      0.00      0.00         0
                 water       0.80      0.35      0.49       402
                  food       0.79      0.50      0.62       688
               shelter       0.77      0.26      0.39       563
              clothing       0.75      0.10      0.17        92
                 money       0.78      

C:\Users\aniyo\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\aniyo\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1439: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
C:\Users\aniyo\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels.
  'precision', 'predicted', average, warn_for)
C:\Users\aniyo\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1439: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels.
  'recall', 'true', average, warn_for)


### 6. Improve your model
Use grid search to find better parameters. 

In [34]:
pipeline.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                   dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                   lowercase=True, max_df=1.0, max_features=None, min_df=1,
                   ngram_range=(1, 1), preprocessor=None, stop_words=None,
                   strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                   tokenizer=<function tokenize at 0x000001CB017B4488>,
                   vocabulary=None)),
  ('tfidf',
   TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)),
  ('clf',
   MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True,
                                                          class_weight=None,
                                                          criterion='gini',
                                                          max_depth=None,
                                                          max_fea

In [35]:
parameters = {
    'clf__estimator__min_samples_leaf': [1, 2],
    'clf__estimator__min_samples_split': [10, 15],
    'clf__estimator__n_estimators': [10, 20]
}

cv = GridSearchCV(pipeline, param_grid = parameters, n_jobs = -1)
cv.fit(X_train, y_train)

C:\Users\aniyo\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


GridSearchCV(cv='warn', error_score='raise-deprecating',
             estimator=Pipeline(memory=None,
                                steps=[('vect',
                                        CountVectorizer(analyzer='word',
                                                        binary=False,
                                                        decode_error='strict',
                                                        dtype=<class 'numpy.int64'>,
                                                        encoding='utf-8',
                                                        input='content',
                                                        lowercase=True,
                                                        max_df=1.0,
                                                        max_features=None,
                                                        min_df=1,
                                                        ngram_range=(1, 1),
                                       

In [38]:
cv.best_params_

{'clf__estimator__min_samples_leaf': 1,
 'clf__estimator__min_samples_split': 10,
 'clf__estimator__n_estimators': 20}

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [37]:
y_pred_new = cv.predict(X_test)
print(classification_report(y_test.values, y_pred_new, target_names = Y.columns.values))

                        precision    recall  f1-score   support

               related       0.83      0.95      0.89      4918
               request       0.80      0.52      0.63      1058
                 offer       0.00      0.00      0.00        28
           aid_related       0.73      0.72      0.73      2669
          medical_help       0.62      0.14      0.23       533
      medical_products       0.79      0.10      0.17       312
     search_and_rescue       0.68      0.09      0.16       163
              security       0.00      0.00      0.00       121
              military       0.83      0.09      0.17       204
           child_alone       0.00      0.00      0.00         0
                 water       0.83      0.36      0.50       402
                  food       0.80      0.68      0.73       688
               shelter       0.81      0.37      0.51       563
              clothing       0.92      0.12      0.21        92
                 money       0.71      

C:\Users\aniyo\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\aniyo\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1439: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
C:\Users\aniyo\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels.
  'precision', 'predicted', average, warn_for)
C:\Users\aniyo\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1439: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels.
  'recall', 'true', average, warn_for)


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [25]:
pipeline = Pipeline([
    ('vect', TfidfVectorizer(tokenizer = tokenize)),
    ('clf', MultiOutputClassifier(AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=1, class_weight = 'balanced'))))
    
])

In [27]:
parameters = {
    'vect__smooth_idf': [True, False],
    'clf__estimator__learning_rate': [0.1, 0.3],
    'clf__estimator__n_estimators': [100, 200]
    
}

cv_ada = GridSearchCV(pipeline, param_grid = parameters, n_jobs = -1)

cv_ada.fit(X_train, y_train)

y_pred = cv_ada.predict(X_test)

print(classification_report(y_test.values, y_pred, target_names = Y.columns.values))

                        precision    recall  f1-score   support

               related       0.94      0.70      0.80      5005
               request       0.56      0.75      0.64      1130
                 offer       0.03      0.33      0.06        30
           aid_related       0.74      0.67      0.70      2770
          medical_help       0.39      0.63      0.48       552
      medical_products       0.25      0.63      0.36       337
     search_and_rescue       0.11      0.56      0.18       190
              security       0.04      0.35      0.07       102
              military       0.38      0.72      0.50       222
           child_alone       0.00      0.00      0.00         0
                 water       0.55      0.88      0.68       431
                  food       0.73      0.83      0.78       733
               shelter       0.57      0.80      0.66       544
              clothing       0.30      0.69      0.42       105
                 money       0.29      

C:\Users\aniyo\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\aniyo\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1439: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
C:\Users\aniyo\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels.
  'precision', 'predicted', average, warn_for)
C:\Users\aniyo\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1439: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels.
  'recall', 'true', average, warn_for)


### 9. Export your model as a pickle file

In [39]:
with open('clf.pkl', 'wb') as f:
    pickle.dump(cv, f)

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.

In [ ]:
from sqlalchemy import create_engine
import nltk
from nltk.corpus import stopwords
import re
import pickle
import numpy as np
import pandas as pd
from nltk.tokenize import word_tokenize
from nltk.stem.wordnet import WordNetLemmatizer
from sklearn.pipeline import Pipeline
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer, TfidfVectorizer
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV
nltk.download(['punkt', 'wordnet', 'stopwords'])


In [ ]:
def load_data(db_name = 'disaster_msg.db', table_name = 'disaster_msg'):
    '''
    This functions reads data from the database and returns X and y
    
    Args:
    db_name : name of the database
    table_name : name of the table
    
    Out:
    X : input messages for the machine learning model
    y : output labels for the input messages
    
    '''
    
    engine = create_engine('sqlite:///'+db_name)
    df = pd.read_sql("SELECT * FROM "+db_name, engine)
    X = df['message']

    y = df.drop(['id', 'message', 'original', 'genre'], axis = 1)
    
    return X, y

def tokenize(text):
    '''
    A function to normalize, tokenize and stem the given string
    
    Input:
    text : A string containing a message that needs to be processed
    
    Output:
    clean_tok : A list of normalized and stemmed tokens from the given string
    '''
    
    # Normalize text : converting it to lowercase and removing punctuation
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())
    
    # Tokenize text
    words = word_tokenize(text)
    tokens = [w for w in words if w not in stopwords.words('english')]
    
    # Lemmatize
    lemmatizer = WordNetLemmatizer()
    clean_tok = []
    for tok in tokens:
        clean_token = lemmatizer.lemmatize(tok).strip()
        clean_tok.append(clean_token)
    
    return clean_tok

def build_model():
    '''
    A function that consists of the Machine Learning Pipeline
    
    Input:
    N/A
    
    Output:
    model : grid search model
    
    '''
    pipeline = Pipeline([
        ('vect', TfidfVectorizer(tokenizer = tokenize)),
        ('clf', MultiOutputClassifier(RandomForestClassifier()))
    ])
    
    parameters = {
        'clf__estimator__min_samples_leaf': [1, 2],
        'clf__estimator__min_samples_split': [10, 15],
        'clf__estimator__n_estimators': [10, 20]
    }

    cv = GridSearchCV(pipeline, param_grid = parameters, n_jobs = -1)
    
    return cv

def train(X, y, model_blueprint):
    '''
    A function that takes in input and output labels and fits to a model
    
    Input:
    X : input training data
    y : output training labels
    model_blueprint : model to fit the training data
    
    Output:
    model: Fitted model
    
    '''
    
    # Create the train_test split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)
    
    # Fit the model
    model = model_blueprint.fit(X_train, y_train)
    
    # Get the test prediction results
    y_pred = model.predict(X_test)
    
    # Display evaluation results
    print(classification_accuracy(y_test.values, y_pred, target_names = y_test.columns.values))
    
    return model
    
    
    
    
def export_model(model):
    '''
    A function to save our model as a pickle file
    
    Input:
    model : target model
    
    Output:
    N/A
    
    '''
    
    with open('clf.pkl', 'wb') as f:
        pickle.dump(model, f)
        
    
def run_pipeline(data_file):
    print("Step 1: Loading Data")
    X, y = load_data(data_file)
    print("Step 2: Building Model")
    model = build_model()
    print("Step 3: Training Model")
    model = train(X, y, model)
    y_pred = model.predict(X_test)
    print('Step 5: Saving Model')
    export_model(model)
    

In [ ]:
if __name__ == "__main__":
    data_file = sys.argv[1]
    run_pipeline(data_file)